# Lab 2: Step Response of an RC Circuit

## Introduction

This lab has two primary learning objectives:

1. helping students learn how to run real-time dynamic systems and
   control experiments using the Arduino/Serial/Python approach
2. helping students learn about transfer functions through
   investigating the step response of an RC circuit

Throughout this course, we will use Arduino microcontrollers in conjunction with Python to run dynamic systems and control experiments.  Students will need to learn how this approach works and what must be done both in Python and on the Arduino to run such tests.

An RC circuit is a fairly simple dynamic system and will be used in this lab to help students begin to think in terms of transfer functions.  The results of an experimental step response will be used to estimate transfer function coefficients.  Experimental results will then be compared to Python simulations.


## Main Steps

- understand the provided Arduino code and how the real-time serial approach works
- build an RC circuit and connect it to the Arduino
- complete the starter Python code so that a step response test is run while Python and the Arduino are communicating over serial
- overlay the experimental step response with the result of Laplace analysis "by hand" as well as simulation results using `control.step_response`
- curve fit the experimental step response data to improve your estimate of $p$

## Part 1: The Arduino/Serial/Python Approach to Dynamic Systems Experiments

There are several challenges associated with running dynamic systems and control experiments using Arduino microcontrollers.  Two important challenges are

1. getting data from the Arduino back to Python and
2. running the experiments at hard, real-time intervals.

In order to run these experiments, Python must communicate with the Arduino using serial communication.  Serial communication does not work well if the Arduino is endlessly and constantly streaming data. So, an approach is needed that starts printing the data to the serial monitor when the test begins and then stops printing after a certain period of time.  When a test is over, the Arduino will stop printing data and go to a menu function where it will wait for the user to tell it to start the next test.

Additionally, it is best if dynamic systems and control experiments are performed at hard, real-time intervals which are best controlled using timer interrupts.  It is important that the Arduino perform its control actions at precise time intervals.  These actions generally include reading sensor values and sending actuator commands.  The time intervals will be controlled by a timer interrupt.  When the interrupt happens, it will set a flag variable to a value of 1 to tell the main `loop` function to perform the actions associated with the next time step in the test.

You will recall that all Arduino scripts must have two required functions: `setup` and `loop`.  Once the `setup` function has been executed, the Arduino continually runs through the `loop` function.  While running a dynamic systems and control test, an Arduino is typically doing one of three things:

1. waiting for the test to start
2. waiting for the next timer interrupt
3. performing the actions associated with one time step of a test

So, each time through the `loop` function, the Arduino must decide if the timer interrupt has happened or if the test is over. If the test is not over, the Arduino continually checks whether or not the timer interrupt has happened.  If the timer interrupt has not yet happened, the Arduino essentially does nothing and goes back to the top of the `loop` function to check again if the timer interrupt has happened.  To accomplish this, most of the `loop` function is inside an `if` statement that only executes once the timer interrupt happens.

Once the timer interrupt happens, the Arduino reads all of the sensor values, sends out all actuator signals, and prints the data for that time step to the serial monitor.  These are the actions that are necessary for each time step of a dynamic systems and control test.

After a certain number of time steps has occurred, where each timer
interrupt corresponds to one time step, the Arduino stops sending data over serial and waits for the next test to start.


## Arduino Code

You should not need to modify the Arduino code for this lab.  You goal is to write Python code that communicates over serial with the Arduino to perform the step response test and get the data back to Python.

Here is a link to the Arduino code:
[https://drive.google.com/file/d/1QQx10phGg0hv7kcT6199vSeUCgut9NtC/view?usp=sharing](https://drive.google.com/file/d/1QQx10phGg0hv7kcT6199vSeUCgut9NtC/view?usp=sharing)

## Part 2: RC Step Response

## First Steps

The first thing your team should do is download the Arduino code and try to understand it.  Then start working on completing the Python code.  Here is the Python starter code:

[https://drive.google.com/file/d/1QS3RuHZcrrGgcvQMfr7Yseo6toAKrH89/view?usp=sharing](https://drive.google.com/file/d/1QS3RuHZcrrGgcvQMfr7Yseo6toAKrH89/view?usp=sharing)

Before you can run tests in Python, you will need to install Dr. Krauss' `serial_utils` module:

`pip install serial_utils`



### Using IPython

Unfortunately, real-time serial tests cannot be run within Jupyter.  You will need to use the `IPython` prompt to run your code.  Open a terminal on a Mac or the Anaconda Prompt in windows and enter the command `ipython3 --pylab`.  You will want to navigate to the folder where you have the code saved first using the `cd` command.

To run your Python script type the command
`run rt_serial_rc_filter_run_test.py`
into the ipython prompt.

## Python Serial Notes

Serial has several quirks beyond getting the baud rate correct and not being able to connect using more than one piece of software at a time (i.e. you must close the Arduino serial monitor before using Python).  Most of the challenges relate to the fact that devices using serial send out their messages without knowing if they have been heard.  Because of this, it is very important to make sure that the devices are sending and reading the correct number of bytes.


### Python commands for reading and writing over serial

#### Reading

- `serial_utils.Read_Byte(ser)`
    - reads a single byte
- `serial_utils.Read_Two_Bytes(ser)`
    - reads a two byte int, but the int is only positive (unsigned) 0-64,000
- `serial_utils.Read_Two_Bytes_Twos_Comp(ser)`
    - reads a two byte int in the range of +/-32,000(ish) (signed)

#### Writing

- `serial_utils.WriteByte(ser, byte)`
- `serial_utils.WriteInt(ser, int)`

## Step Response Testing

<img src="https://drive.google.com/uc?id=1RGNnAZyAhFgZZgN11-3Tfen_nabwZ6uQ" width=400px>

A schematic of an RC filter is shown above.
In order to run a step response test, you will need to modify the
Python code from Part 1 to communicate with the Arduino over serial.


## Saving Your Data

Once you have successfully run an RC step response test, save that data to a CSV or TXT file before you start curve fitting.  This will allow you to work on curve fitting without having to have your Arduino attached and running a new test each time.  `numpy` has a `savetxt` function that works much like `loadtxt`.  Type `help(np.savetxt)` to learn more.


## Curve Fitting

Once you have the step response data, use `scipy.optimize.fmin` to
curve fit it.  In order to do this, you will need to create a function that returns $v_{out}$ for a 5V step input.  First, find the transfer function of the circuit

$$G(s) = \frac{V_{out}(s)}{V_{in}(s)}$$

We found the transfer function in lecture.  Verify the transfer
function we found in lecture using Kirchoff's laws, Ohm's law, and the
voltage/current relationship of a capacitor.  Once you have verified
the transfer function, write a function that finds the step response
with a 5V input.  You ought to be able to modify code from lab 1 to
create the function you need for curve fitting.


## optimize.fmin Example

A Jupyter notebook example has been created to introduce curve fitting in Python based on the function `scipy.optimize.fmin`.  The example requires a `csv` data file.

The Jupyter notebook can be downloaded from here:

[https://drive.google.com/open?id=1bx1yoDHjIefIYuyvU2IVFaefm2wi0oKm](https://drive.google.com/open?id=1bx1yoDHjIefIYuyvU2IVFaefm2wi0oKm)

And the `csv` file can be downloaded from here:

[https://drive.google.com/open?id=19Yi4OjWN82qEPAnp8sW6Qd1u4YR4tSvE](https://drive.google.com/open?id=19Yi4OjWN82qEPAnp8sW6Qd1u4YR4tSvE)

### Curve Fitting Tips

In order to fit a curve to the step response using `optimize.fmin`, you need a function that takes the unknown transfer function coefficients (i.e. $p$ in this case) and returns the step response.  This is very similar to what we did in lab 1 except that we need to find the step response at the same points in time where we have experimental data.  There are two options for doing this:

1. Solve the step response symbolically with $p$ as a variable and then just have your `mymodel` function use that symbolic expression
2. Pass in your time array as the second input to `control.step_response`

Option 2 is in some ways easier but comes with one additional
challenge: `control.step_response` expects the time vector to have
perfect time steps that are exactly the same.  Essentially, it is expecting that your time vector was created using `np.arange`.  So, you need to find a way to create a fake, perfect time vector that is as close as possible to your real, experimental time vector.  Find the average time step `dt` from your experimental time vector and use it along with `np.arange` to create a fake time vector that has the same length as the experimental time vector.

Note that in Python, functions always have read-only access to all of
the variables in the notebook or Python file where they reside.  So,
you need to pass your fake time vector as a second input to
`control.step_response`, but it does not need to be an input to the
`mymodel` function.  All you need to do is define the fake time vector
somewhere in the notebook.  The only input to `mymodel` should be a
list of unknown coefficients, `[p]`.


## Step Response Simulation

You are required to compare your experimental results to a simulation
of the step response using either `control.step_response` or
`control.forced_response` from the Python `control` module.  You will
need the transfer function of the system to do this.  To show your
results, overlay data from your experimental step response on the same
plot as simulation results based on your optimized coefficient values.
Also, verify that the step response from the `control` module agrees with
your derivation for $v_{out}(t)$ for the step input.


# Comprehension Questions

### CQ1: step response

What steps are necessary to perform step response analysis "by hand"?  How do you do the same steps using Python to verify your answer?

### CQ2: optimize.fmin

What does `optimze.fmin` actually do?  What is the scalar value returned by the cost function?  